In [5]:
import pandas as pd
import spacy 

nlp = spacy.load("en_core_web_sm")

In [7]:
df = (pd.read_csv("Questions.csv", nrows=2_000_000, 
                  encoding="ISO-8859-1", usecols=['Title', 'Id']))

titles = [_ for _ in df.loc[lambda d: d['Title'].str.lower().str.contains("go")]['Title']]

In [8]:
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [11]:
%%time

def has_golang(doc):
    for t in doc:
        if t.lower_ in ["go", "golang"]:
            if t.pos_ == "NOUN":
                return True 
    return False

g = (doc for doc in nlp.pipe(titles) if has_golang(doc))
[next(g) for i in range(30)]

CPU times: total: 13 s
Wall time: 13 s


[Can I append an Ajax requestXML object to my document tree all in one go?,
 Deploying multiple Java web apps to Glassfish in one go,
 Removing all event handlers in one go,
 How do I disable multiple listboxes in one go using jQuery?,
 Is there a way to find a specific file and then change into the directory containing it in one go?,
 SOAPUI & Groovy Scripts, executing multiple SQL statements in one go,
 Import large chunk of data into Google App Engine Data Store at one go,
 How many records can be loaded into Salesforce using Apex Data Loader in one go?,
 Reading utf8-encoded data from a connection, using Go,
 Checking if a channel has a ready-to-read value, using Go,
 How can I run multiple inserts with NHibernate in one go?,
 Letting go of auto_ptr,
 Saving all nested form objects in one go,
 Generating Random Numbers in Go,
 How do I allocate memory for an array in the go programming language?,
 Any tricks to export Go docs to be served statically?,
 What's wrong with the followi

In [12]:
df_tags = pd.read_csv("Tags.csv")
go_ids = df_tags.loc[lambda d: d['Tag'] == 'go']['Id']

def has_go_token(doc):
    for t in doc:
        if t.lower_ in ['go', 'golang']:
            if t.pos_ != 'VERB':
                return True
    return False

all_go_sentences = df.loc[lambda d: d['Id'].isin(go_ids)]['Title'].tolist()
detectable = [d.text for d in nlp.pipe(all_go_sentences) if has_go_token(d)]

non_detectable = (df
                  .loc[lambda d: ~d['Id'].isin(go_ids)]
                  .loc[lambda d: d['Title'].str.lower().str.contains("go")]
                  ['Title']
                  .tolist())

non_detectable = [d.text for d in nlp.pipe(non_detectable) if has_go_token(d)]

len(all_go_sentences), len(detectable), len(non_detectable)

(1858, 879, 110)

In [13]:
model_name = "en_core_web_sm"
model = spacy.load(model_name, disable=["ner"])

def has_go_token(doc):
    for t in doc:
        if t.lower_ in ["go", "golang"]:
            if t.pos_ != "VERB":
                return True
    return False

method = "not-verb-but-pobj"

correct = sum(has_go_token(doc) for doc in model.pipe(detectable))
wrong = sum(has_go_token(doc) for doc in model.pipe(non_detectable))
precision = correct/(correct + wrong)
recall = correct/len(detectable)
accuracy = (correct + len(non_detectable) - wrong)/(len(detectable) + len(non_detectable))

f"{precision},{recall},{accuracy},{model_name},{method}"

'0.8887765419615774,1.0,0.8887765419615774,en_core_web_sm,not-verb-but-pobj'